In [17]:
import requests
import json
import pandas as pd
retList = []


In [28]:

#호출용 파라미터 지정
def getReviewParam(place, theme, pageNum = 1):
    
    
    return {
    "operationName": "getVisitorReviews",
    "variables": {
      "input": {
        "businessId": place['id'],
        "businessType": "restaurant",
        "item": "0",
        "page": pageNum,
        "size": 40,
        "includeContent": True,
        "getUserStats": True,
        "includeReceiptPhotos": True,
        "cidList": place['categoryPath'][0],
        "theme": theme
      },
      "id": place['id']
    },
  "query": "query getVisitorReviews($input: VisitorReviewsInput) {\n  visitorReviews(input: $input) {\n    items {\n      id\n      rating\n      author {\n        id\n        nickname\n        from\n        imageUrl\n        objectId\n        url\n        review {\n          totalCount\n          imageCount\n          avgRating\n          __typename\n        }\n        theme {\n          totalCount\n          __typename\n        }\n        __typename\n      }\n      body\n      thumbnail\n      media {\n        type\n        thumbnail\n        class\n        videoId\n        videoOriginSource\n        __typename\n      }\n      tags\n      status\n      visitCount\n      viewCount\n      visited\n      created\n      reply {\n        editUrl\n        body\n        editedBy\n        created\n        replyTitle\n        isReported\n        isSuspended\n        __typename\n      }\n      originType\n      item {\n        name\n        code\n        options\n        __typename\n      }\n      language\n      highlightOffsets\n      apolloCacheId\n      translatedText\n      businessName\n      showBookingItemName\n      showBookingItemOptions\n      bookingItemName\n      bookingItemOptions\n      votedKeywords {\n        code\n        iconUrl\n        iconCode\n        displayName\n        __typename\n      }\n      userIdno\n      isFollowing\n      followerCount\n      followRequested\n      loginIdno\n      receiptInfoUrl\n      __typename\n    }\n    starDistribution {\n      score\n      count\n      __typename\n    }\n    hideProductSelectBox\n    total\n    showRecommendationSort\n    itemReviewStats {\n      score\n      count\n      itemId\n      starDistribution {\n        score\n        count\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"
  }



#카테고리 현황 조회용 파라미터
def getStatParam(place):
    return {
  "operationName": "getVisitorReviewStats",
  "variables": {
    "businessType": "restaurant",
    "id": place['id'],
    "itemId": "0"
  },
  "query": "query getVisitorReviewStats($id: String, $itemId: String, $businessType: String = \"place\") {\n  visitorReviewStats(input: {businessId: $id, itemId: $itemId, businessType: $businessType}) {\n    id\n    name\n    apolloCacheId\n    review {\n      avgRating\n      totalCount\n      scores {\n        count\n        score\n        __typename\n      }\n      starDistribution {\n        count\n        score\n        __typename\n      }\n      imageReviewCount\n      authorCount\n      maxSingleReviewScoreCount\n      maxScoreWithMaxCount\n      __typename\n    }\n    analysis {\n      themes {\n        code\n        label\n        count\n        __typename\n      }\n      menus {\n        label\n        count\n        __typename\n      }\n      votedKeyword {\n        totalCount\n        reviewCount\n        userCount\n        details {\n          category\n          code\n          iconUrl\n          iconCode\n          displayName\n          count\n          previousRank\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    visitorReviewsTotal\n    ratingReviewsTotal\n    __typename\n  }\n}\n"
}



# {
#   "operationName": "getVisitorReviewStats",
#   "variables": {
#     "businessType": "restaurant",
#     "id": "1813392509",
#     "itemId": "0"
#   },
#   "query": "query getVisitorReviewStats($id: String, $itemId: String, $businessType: String = \"place\") {\n  visitorReviewStats(input: {businessId: $id, itemId: $itemId, businessType: $businessType}) {\n    id\n    name\n    apolloCacheId\n    review {\n      avgRating\n      totalCount\n      scores {\n        count\n        score\n        __typename\n      }\n      starDistribution {\n        count\n        score\n        __typename\n      }\n      imageReviewCount\n      authorCount\n      maxSingleReviewScoreCount\n      maxScoreWithMaxCount\n      __typename\n    }\n    analysis {\n      themes {\n        code\n        label\n        count\n        __typename\n      }\n      menus {\n        label\n        count\n        __typename\n      }\n      votedKeyword {\n        totalCount\n        reviewCount\n        userCount\n        details {\n          category\n          code\n          iconUrl\n          iconCode\n          displayName\n          count\n          previousRank\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    visitorReviewsTotal\n    ratingReviewsTotal\n    __typename\n  }\n}\n"
# }




def placeProc(place):
    #장소별 ret 초기화
    placeDict = {}

    placeDict['id'] =  place['id']
    placeDict['name'] =  place['name']
    placeDict['contents'] =  []
    
    statParam = getStatParam(place)

    # 리뷰/테마를 읽어오기 위한 api 정의
    review_url = "https://pcmap-api.place.naver.com/graphql"
    review_headers = {
        "referer": "https://pcmap.place.naver.com/",
        "content-type": "application/json",
        "accept": "*/*",
        "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2"
    }

    # 테마 목록 가져오기 
    stat_response = requests.post(review_url, headers=review_headers, data=json.dumps(statParam))
    print('stat request', place['name'], stat_response)
    if stat_response.status_code != 200:        
        return placeDict;
    stat_json = json.loads(stat_response.text)

    # 테막 목록 확인
    themes = stat_json['data']['visitorReviewStats']['analysis']['themes']
    # 테마별 리뷰를 불러온다.
    placeDict['contents'] = []
    for theme in themes:
        # total은 제외한다
        if theme['code'] in ['taste','service','price']:
            nextPage = True
            pageNum = 1
            print (pageNum, nextPage)
            while nextPage:
              print (pageNum, nextPage)
              review_param = getReviewParam(place,theme['code'], pageNum)
              review_response = requests.post(review_url, headers=review_headers, data=json.dumps(review_param))
              if review_response.status_code != 200: 
                  continue;
              review_json = json.loads(review_response.text)
              reviewBody = review_json['data']['visitorReviews']['items']
              print(theme['code'], len(reviewBody))
              placeDict['contents'].extend(list(map(lambda x: {'div': theme['code'], 'cont':x['body']} , reviewBody)))
              if len(reviewBody)==40:
                  pageNum = pageNum + 1
              else:
                  nextPage = False
                  break
            # #반환된 review 건수가 40건인 경우 page 2 한번 더 호출한다.
            # if len(reviewBody)==40:
            #     review_param = getReviewParam(place, theme['code'], 2)
            #     review_response = requests.post(review_url, headers=review_headers, data=json.dumps(review_param))
            #     if review_response.status_code != 200: 
            #         continue;
            #     review_json = json.loads(review_response.text)
            #     reviewBody = review_json['data']['visitorReviews']['items']
            #     print(theme['code'], len(reviewBody))
            #     placeDict['contents'].extend(list(map(lambda x: {'div': theme['code'], 'cont':x['body']} , reviewBody)))

    return placeDict
    
    

In [29]:
url = 'https://map.naver.com/v5/api/search'

headers = {
    'authority': 'map.naver.com',
    'content-type': 'application/json',
    "accept": "*/*",
    "user-agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.2 (KHTML, like Gecko) Chrome/22.0.1216.0 Safari/537.2"
}

### 음식점 목록을 가져온다 (현재 위치 위경도값:searchCoord, query : 검색어)
### 호출건수 : displayCount : 100
params = {
    'lang': 'ko',
    'caller': 'pcweb',
    'type': 'place',
    #'searchCoord': '37.272429000000116,127.13596284980378', #용인
    'query': '평가옥분당점',
    'page': 1,
    'displayCount': 1,
    'isPlaceRecommendationReplace': 'false',
}


# clientX: 127.1275729
# clientY: 37.272429
# x: 127.10589175393739
# y: 37.36958999999972
# bounds: 127.09966902902255;37.363723468708145;127.12069754769959;37.37528555368495


In [30]:

response = requests.get(url, headers=headers, params=params)

result = json.loads(response.text)

#호출결과에서 장소에 대한 목록만 읽어온다
placeList = result['result']['place']['list']

In [31]:
len(placeList)

1

In [32]:
# 길이 확인 및 첫 음식점 확인
print(len(placeList))
retList = []
placeList[0]
print(placeList[0]['id'])
cnt = 0
for place in placeList:
    cnt+=1
    # if cnt<50: continue
    retList.append(placeProc(place))
    
    
# retList.append(placeProc(placeList[1]))
# retList.append(placeProc(placeList[2]))

1
12867892


In [14]:
retJson = json.dumps(retList)

from datetime import datetime
nowstr = datetime.now().strftime('%Y.%m.%d.%H%M%S') # 년.월.일 - 시간
with open("./retJson_{0}.json".format(nowstr), 'w', encoding='utf8') as outfile:
    json.dump(retList, outfile, ensure_ascii=False)

In [2]:
import os

# 폴더 경로
folder_path = "./"

# JSON 파일 리스트
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

# JSON 파일 읽기 및 병합
merged_json = []
for file_name in json_files:
    print(file_name)
    with open(os.path.join(folder_path, file_name), 'r') as file:
        json_data = json.load(file)
        merged_json.extend(json_data[::-1])

nowstr = datetime.now().strftime('%Y.%m.%d.%H%M%S') # 년.월.일 - 시간

# 병합된 JSON 파일 저장
with open("./merged_{0}.json".format(nowstr), 'w', encoding='utf8') as file:
    json.dump(merged_json, file, ensure_ascii=False)

merged_2023.05.07.132015.json


NameError: name 'json' is not defined

In [30]:
merged_json

In [31]:
len(merged_json)

505

In [35]:
cnt = 0
last_json = []
for place in merged_json:
    if cnt>1: continue;
    if len(place['contents'])>0:
        # 'div' 키의 값이 'taste', 'service', 'price' 인 데이터만 저장할 리스트
        last_json.extend(list(filter(lambda x: x.get('div') in ['taste', 'service', 'price'], place['contents'])))

    

In [36]:
last_json

[{'div': 'taste',
  'cont': '번화가에서 좀만 사이드로 들어가면 나오는 숨겨진 맛집 느낌이라 좋아요 좌석 간격도 널찍하고 프라이빗한 느낌이라 샌드위치 든든하게 먹으면서 휴식 취하는 느낌 굿 마라떼 처음 먹어보는데 맛있고 샌드위치도 신선해요'},
 {'div': 'taste',
  'cont': '읍천리382 기흥점💘 \n사장님 친절하시고 디저트랑 음료도 맛있어요. 종류도 다양해서 골라먹는 재미도 있네요. 맛도 맛있는데 먹고 싶은 메뉴가 많아서 다음에 또 방문하려고요!!! 😋'},
 {'div': 'taste',
  'cont': '자주 왔던 곳인데 동네에 이런 카페가 있다는게 너무 좋아요!! 인테리어도 깔끔하고 커피도 맛있고 콘치즈 토스트 특히 맛있어요!! '},
 {'div': 'taste',
  'cont': '동네에 조용하고 분위기 좋고 좌석 편한 카페가 생겨서 좋아요!! 사장님도 너무 친절하시고 커피랑 샌드위치도 맛있어요ㅎㅎㅎ '},
 {'div': 'taste',
  'cont': '인테리어가 특색있고 메뉴 하나하나 다 너무 맛있어서 자꾸 찾게되는 곳이예요🥰\n터질듯한 샌드위치와 페스츄리와플 너무 맛있어요!!\n특히 앙버터 페스츄리와플은 진짜 세상사람들 다 먹어봤음 좋겠어요..❣️\n가성비 너무 좋고 맛있는 읍천리 앞으로도 쭉~~ 애정할 예정이예요 희희'},
 {'div': 'taste',
  'cont': '음료도 맛있구 샌드위치도 맛있어용⭐️\n집이랑 가까워서 가끔오는 분위기맛집⭐️\n사장님듀 너무 친절하세용⭐️'},
 {'div': 'taste', 'cont': '쉬림프 샌드위치에 아아 조졌습니다.\n맛있었어요 ! 그리고 사장님 완전 친절 😍'},
 {'div': 'taste',
  'cont': '샌드위치랑 와플 너무 맛있어요 ... 아메리카노랑 조합 최고에요 읍천리는 진짜 먹어도 먹어도 맛있네요 !!!!'},
 {'div': 'taste',
  'cont': '토피넛라떼랑 마라떼 주문했는데 토피넛라떼는 달달하고 너무 맛있

In [37]:
df = pd.DataFrame(last_json)

In [38]:
df

,div,cont
0,taste,번화가에서 좀만 사이드로 들어가면 나오는 숨겨진 맛집 느낌이라 좋아요 좌석 간격도 ...
1,taste,읍천리382 기흥점💘 \n사장님 친절하시고 디저트랑 음료도 맛있어요. 종류도 다양해...
2,taste,자주 왔던 곳인데 동네에 이런 카페가 있다는게 너무 좋아요!! 인테리어도 깔끔하고 ...
3,taste,동네에 조용하고 분위기 좋고 좌석 편한 카페가 생겨서 좋아요!! 사장님도 너무 친절...
4,taste,인테리어가 특색있고 메뉴 하나하나 다 너무 맛있어서 자꾸 찾게되는 곳이예요🥰\n터질...
...,...,...
6499,service,사장님이 친절하고 \n음식도 정성껏 만드신게 느껴져서 좋았어요\n다른 메뉴들도 맛있...
6500,service,견과류이서 기름 쩐내 났어요 ㅠㅠ\n재료 신경 써주시면 더 좋을 거 같아요
6501,service,"재료가 신선하고, 양이 푸짐해요. 배부르게, 맛있게 먹었어요. 무엇보다 사장님이 넘..."
6502,service,친절하시구 넘 맛있어용 자주 올거예열😊


In [39]:
df.to_csv('review_fin.csv')